<a id='intro'></a>
## Introduction
> This notebook is for training the 6sense data through different models (such as Random Froest, KNN, XGBoost and so on) and find the best fit model through several variables including call disposition, best day of week to call and best hour to call, predicting the success or not if a customer would pick up phone calls so that the company can get engagement with prospective customers.

## Imports

In [220]:
# import libraries
import pandas as pd
import re
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split

import sklearn

import seaborn as sns
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split,cross_val_score,KFold,cross_val_predict
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score,confusion_matrix,precision_recall_curve,roc_curve
from sklearn.feature_selection import RFE
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, KBinsDiscretizer, FunctionTransformer

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import ExtraTreesClassifier,RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron, SGDClassifier, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
# import xgboost as xgb

In [221]:
#import data
events = pd.read_csv("../data/6Sense/events.csv") #id not unique
calls = pd.read_csv("../data/6Sense/calls.csv") #id not unique
companies = pd.read_csv("../data/6Sense/companies.csv") #id unique
people = pd.read_csv("../data/6Sense/people.csv")# contact id unique
opportunities = pd.read_csv("../data/6Sense/opportunities.csv")# company id not unique
events = events.rename(columns={"date": "event_date"})
calls = calls.rename(columns={"timestamp": "call_time"})
del calls['date']

In [222]:
clean_activity_name = []
for index, row in events.iterrows():

SyntaxError: unexpected EOF while parsing (<ipython-input-222-27eca767f6c2>, line 2)

In [231]:
cleaned_activity_name = []
for index, row in events.iterrows():
    if row["activity_action"] == "view":
        if ".com" in row["activity_name"]:
            temp = row["activity_name"].split("/")[:-1]
            if len(temp) >= 1: 
                cleaned_activity_name.append(temp[-1])
            else:
                cleaned_activity_name.append("unknown")
        else:
            cleaned_activity_name.append("unknown")
    elif row["activity_action"] == "fill_out":
        cleaned_activity_name.append("filled "+ row["activity_name"].split("-")[0])
    elif row["activity_action"] == "open" or row["activity_action"] == "click":
        temp = row["activity_name"].split("-")
        if len(temp) >= 1:
            if re.match("(?:\d.*?){2,}", temp[0]) and "-" in row["activity_name"]:
                temp = temp[1:]
            if 'webinar' in temp or ' webinar ' in temp:
                for index, x in enumerate(temp): 
                    if "." in x:
                        temp[index] = x.split(".")[0]
                temp = [x for x in temp if "series" not in x][1:]
            if 'website ' in temp or ' partner webinar ' in temp:
                for index, x in enumerate(temp): 
                    if "." in x:
                        temp[index] = x.split(".")[0]
            if ' field event ' in temp or ' tradeshow ' in temp:
                temp = temp[:2][1:]
            if 'tradeshow field event' in temp: 
            cleaned_activity_name.append("-".join(temp))
        else:
            cleaned_activity_name.append("unknown")
    elif row["activity_action"] == "attended_in_person":
        temp = (row["activity_name"].split("-"))
        if len(temp) > 1:
            temp = "-".join(temp[1:3])
        else:
            temp = row["activity_name"]
        cleaned_activity_name.append(temp)
    elif row["activity_action"] == "filled_out_form":
        temp = (row["activity_name"].split("-"))
        if len(temp) >= 1:
            if re.match("(?:\d.*?){2,}", temp[0]) and "-" in row["activity_name"]:
                temp = temp[1:]
        cleaned_activity_name.append("-".join(temp))
    else:
        cleaned_activity_name.append("unknown")
            
(cleaned_activity_name)

['platform',
 'unknown',
 'filled standard responsive form  ',
 'siriusdecisions & quorum',
 'siriusdecisions & quorum',
 ' personalize at scale-invite last chance',
 '2019_02_email_talkingsense_prospect.2019_02_email_talkingsense1_prospect',
 'our-partners',
 ' personalize & prioritize direct gifting',
 ' personalize & prioritize direct gifting',
 'meet-the-senseis-uncovering-demand-webinar',
 ' abm fast track',
 ' uncovering demand',
 ' uncovering demand',
 ' uncovering demand',
 ' uncovering demand',
 ' uncovering demand',
 '6sense | follow up (1563911953.0)',
 'website - free trial request- report request confirmation',
 ' abm search & display',
 ' 6sense breakthrough ',
 'careers',
 ' b2bsmx ',
 ' abm search & display-invite last chance',
 ' abm search & display-invite last chance',
 'resources',
 'filled standard responsive form  ',
 ' partner webinar - pedowitz techtalk',
 ' tradeshow - hypergrowth ',
 ' fleet week ',
 'women-in-b2b-marketing-empowerment-dinner-series-austin',
 

In [225]:
list(events["activity_action"].unique())

['view', 'fill_out', 'open', 'click', 'attended_in_person', 'filled_out_form']

In [240]:
list(events[events["activity_action"] == "attended_in_person"]["activity_name"])

['2019_11 - tradeshow - hypergrowth - nov 18',
 '2019_11 - tradeshow - topo open house - nov 20-21',
 '2019_11 - tradeshow - hypergrowth - nov 18',
 '2019_02-field event-wine down @ b2bmx',
 '2019_11 - tradeshow - topo open house - nov 20-21',
 '2019_11 - tradeshow - topo open house - nov 20-21',
 '2019_11 - tradeshow - hypergrowth - nov 18',
 '2019_02-field event-wine down @ b2bmx',
 '2019_03- field event - oraclemce - march18-20',
 '2019_11 - tradeshow - topo open house - nov 20-21',
 '2019_05_21 - partner webinar - pedowitz techtalk',
 '2019_11 - tradeshow - topo open house - nov 20-21',
 '2019_03- field event - oraclemce - march18-20',
 '2019_11 - tradeshow - hypergrowth - nov 18',
 '2019_08 -field event-quince dinner - aug 21',
 '2019_03 - field event - cmo connect denver - march_3-5',
 '2019_11 - tradeshow - topo open house - nov 20-21',
 '2019_11 - tradeshow - hypergrowth - nov 18',
 '2019_11 - tradeshow - topo open house - nov 20-21',
 '2019_11 - tradeshow - hypergrowth - nov 1

In [227]:
events[events["contact_id"]=="cc8679776bc960a48"]

,event_date,activity_action,activity_type,activity_name,contact_id,talking_points
88,2019-10-16,open,email_open,2019_10_30 - webinar -6sense+ series 03 - pers...,cc8679776bc960a48,personalize at scale
199,2019-10-31,open,email_open,2019_11_05 - webinar -6sense+ series 04 - pers...,cc8679776bc960a48,personalize web with account-based data
245,2019-09-27,open,email_open,2019_12 - field event - 6sense breakthrough - ...,cc8679776bc960a48,6sense breakthrough
344,2019-11-22,open,email_open,2019_11 - tradeshow - hypergrowth - nov 18.03_...,cc8679776bc960a48,hypergrowth
504,2019-10-14,open,email_open,2019_10_15 - webinar -6sense+ series 02 - abm ...,cc8679776bc960a48,abm search & display-invite last chance
727,2019-09-30,open,email_open,2019_10_01 - webinar - 6sense+ series 01 - per...,cc8679776bc960a48,personalize & prioritize direct gifting-invit...
796,2019-11-21,attended_in_person,attended_event,2019_11 - tradeshow - hypergrowth - nov 18,cc8679776bc960a48,tradeshow - hypergrowth
951,2019-11-04,open,email_open,2019_11_05 - webinar -6sense+ series 04 - pers...,cc8679776bc960a48,personalize web with account-based data-invit...


In [230]:
events["talking_points"] = cleaned_activity_name
talking_points = events.groupby("contact_id").agg(lambda x: list(x))["talking_points"].reset_index()
talking_points.to_csv("../data/talking_points.csv", index=False)

In [232]:
events

,event_date,activity_action,activity_type,activity_name,contact_id,talking_points
0,2019-06-26,view,known_web_visit,6sense.com/platform/,c0655569bba876988,platform
1,2019-09-05,view,known_web_visit,empowered-cmo-2019,68a70ba6b8c0497b8,unknown
2,2019-06-05,fill_out,form_fill,standard responsive form - webinar,54589bb60465a5cc6,filled standard responsive form
3,2019-03-14,open,email_open,2019_03-webinar-siriusdecisions & quorum.03_in...,a08776a8b76688868,siriusdecisions & quorum
4,2019-03-04,open,email_open,2019_03-webinar-siriusdecisions & quorum.01_in...,b47550b45b595c87b,siriusdecisions & quorum
...,...,...,...,...,...,...
2010,2019-06-25,open,email_open,2019_06_26 - webinar - sensei series 03 - unco...,b984ac86ba8ab0576,uncovering demand-invite last chance
2011,2019-10-09,click,email_click,2019_10_15 - webinar -6sense+ series 02 - abm ...,b46976a47c5b7777b,abm search & display
2012,2019-09-28,open,email_open,2019_12 - field event - 6sense breakthrough - ...,694a8995cca7c4456,6sense breakthrough
2013,2019-03-18,open,email_open,2019_03_email_pardot_webinar_promo.2019_03_ema...,c450c67950474c8b0,2019_03_email_pardot_webinar_promo.2019_03_ema...
